In [1]:
import pygrib
import netCDF4 as nc
import numpy as np
import os, glob
import datetime as dt

In [2]:
nc_data = nc.Dataset('../Interpolation/allDataInterpolated.nc')

In [3]:
vars_nc = []
for v in nc_data.variables:
    if v != 'time' and v!= 'level' and v!='lat' and v!= 'lon':
        vars_nc.append(v)
vars_nc

['air', 'uwnd', 'hgt', 'vwnd', 'omega', 'rhum']

In [4]:
def netcdf_var(grib_var):
    if grib_var == 'Temperature':
        return 'air'
    elif grib_var == 'U component of wind':
        return 'uwnd'
    elif grib_var == 'V component of wind':
        return 'vwnd'
    elif grib_var == 'Relative humidity':
        return 'rhum'
    elif grib_var == 'Geopotential height':
        return 'hgt'
    elif grib_var == 'Vertical velocity':
        return 'omega'
    else:
        return None

In [5]:
nc_levels = nc_data['level'][:]
nc_levels.data

array([1000.,  925.,  850.,  700.,  600.,  500.,  400.,  300.,  250.,
        200.,  150.,  100.,   70.,   50.,   30.,   20.,   10.],
      dtype=float32)

In [6]:
def get_indx_level(level, nc_levels):
    return np.where(nc_levels == level)[0][0]

In [7]:
1 in nc_levels

False

In [8]:
for i, level in enumerate(nc_levels):
    print(i,level)

0 1000.0
1 925.0
2 850.0
3 700.0
4 600.0
5 500.0
6 400.0
7 300.0
8 250.0
9 200.0
10 150.0
11 100.0
12 70.0
13 50.0
14 30.0
15 20.0
16 10.0


In [9]:
def update_message(grb, ncData):
    temp_values = np.array(grb.values, copy = True)
    temp_values[:] = ncData[:]
    grb['values'] = temp_values[:]
    return grb.tostring()

In [10]:
def str2date(date:str):
    return dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

In [11]:
def utf2date(utfdate):
    return np.array(nc.num2date(nc_data['time'][:], nc_data['time'].units))

In [12]:
def calculateIndxNetcdf(i_date:str, dates):
    return np.argmin(np.abs(dates - str2date(i_date)))

In [13]:
dir_gribs = os.listdir('../../workspace/wrf-test/grib-data/')

In [14]:
dir_gribs

['fnl_20220417_00_00.grib2',
 'fnl_20220417_06_00.grib2',
 'fnl_20220417_12_00.grib2',
 'fnl_20220417_18_00.grib2',
 'fnl_20220418_00_00.grib2',
 'fnl_20220418_06_00.grib2',
 'fnl_20220418_12_00.grib2',
 'fnl_20220418_18_00.grib2',
 'fnl_20220419_00_00.grib2',
 'fnl_20220419_06_00.grib2',
 'fnl_20220419_12_00.grib2',
 'fnl_20220419_18_00.grib2',
 'fnl_20220420_00_00.grib2',
 'fnl_20220420_06_00.grib2',
 'fnl_20220420_12_00.grib2',
 'fnl_20220420_18_00.grib2',
 'fnl_20220421_00_00.grib2',
 'fnl_20220421_06_00.grib2',
 'fnl_20220421_12_00.grib2',
 'fnl_20220421_18_00.grib2',
 'fnl_20220422_00_00.grib2',
 'fnl_20220422_06_00.grib2',
 'fnl_20220422_12_00.grib2',
 'fnl_20220422_18_00.grib2',
 'fnl_20220423_00_00.grib2',
 'fnl_20220423_06_00.grib2',
 'fnl_20220423_12_00.grib2',
 'fnl_20220423_18_00.grib2',
 'fnl_20220424_00_00.grib2',
 'fnl_20220424_06_00.grib2',
 'fnl_20220424_12_00.grib2',
 'fnl_20220424_18_00.grib2',
 'fnl_20220425_00_00.grib2',
 'fnl_20220425_06_00.grib2',
 'fnl_20220425

# grib2

In [15]:
dates = [i[4:8]+'-'+i[8:10]+'-'+i[10:15].replace('_',' ')+':00:00' for i in dir_gribs]
nc_dates = utf2date(nc_data['time'][:])

In [16]:
def injection():    
    for i, filepath in enumerate(glob.glob('../../workspace/wrf-test/grib-data/*.grib2')):
        idx = calculateIndxNetcdf(dates[i], nc_dates)
        grbs = pygrib.open(filepath)
        messages = []
        for grb in grbs:
            if netcdf_var(grb.name) != None and grb.level in nc_levels and grb.typeOfLevel == 'isobaricInhPa':
                messages.append(update_message(grb, nc_data[netcdf_var(grb.name)][idx, get_indx_level(grb.level, nc_levels)]))
            else:
                messages.append(grb.tostring())
        grbs.close()
        os.remove(filepath)
        outgrb = open(filepath,'wb')
#         outgrb = open(f'./../../workspace/wrfda-injected/{dir_gribs[i]}','wb')
        for msg in messages:
            outgrb.write(msg)
        outgrb.close()
        print(f'{dir_gribs[i]} injected and saved on: {filepath}')
    print('Injection done!')

In [17]:
injection()

fnl_20220417_00_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220417_00_00.grib2
fnl_20220417_06_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220417_06_00.grib2
fnl_20220417_12_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220417_12_00.grib2
fnl_20220417_18_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220417_18_00.grib2
fnl_20220418_00_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220418_00_00.grib2
fnl_20220418_06_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220418_06_00.grib2
fnl_20220418_12_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220418_12_00.grib2
fnl_20220418_18_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220418_18_00.grib2
fnl_20220419_00_00.grib2 injected and saved on: ../../workspace/wrf-test/grib-data/fnl_20220419_00_00.grib2
fnl_20220419_06_00.grib2 inj